In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


# Import Libraries

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
import nltk.corpus
import re
import string
from nltk.corpus import stopwords
import gensim

# Load Data

In [ ]:
train_data=pd.read_csv('/content/train.csv')
test_data=pd.read_csv('/content/test.csv')
submission=pd.read_csv('/content/sample_submission.csv')

# Data preprocessing

In [ ]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_data=train_data.drop(["keyword","location"],axis=1)
test_data=test_data.drop(["keyword","location"],axis=1)

train_data.head(2)

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1


In [ ]:
type(train_data)

pandas.core.frame.DataFrame

In [ ]:
total_data= pd.concat([train_data,test_data],axis=0,ignore_index=True)

In [ ]:
total_data

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,Forest fire near La Ronge Sask. Canada,1.0
2,5,All residents asked to 'shelter in place' are ...,1.0
3,6,"13,000 people receive #wildfires evacuation or...",1.0
4,7,Just got sent this photo from Ruby #Alaska as ...,1.0
...,...,...,...
10871,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,NaN
10872,10865,Storm in RI worse than last hurricane. My city...,NaN
10873,10868,Green Line derailment in Chicago http://t.co/U...,NaN
10874,10874,MEG issues Hazardous Weather Outlook (HWO) htt...,NaN


In [ ]:
# handel urls
def remove_url(text):
    url=re.compile(r'https?://S+|www\.\S+')
    return url.sub(r'',text)
def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

In [ ]:
total_data['text']=total_data['text'].apply(lambda x:remove_url(x))
total_data['text']=total_data['text'].apply(lambda x:remove_html(x))

In [ ]:
# convert to lower alphabates
for i in range(len(total_data)):
    total_data['text'][i].lower()

In [ ]:
stop_words=set(stopwords.words("english"))
print(stop_words)

{'has', 'does', 'any', 'them', 'then', "haven't", 'each', 'their', 'that', 'below', 'in', 'needn', "isn't", 'she', 'until', 'ain', 'against', 'for', 'after', 'those', 'only', 'what', 'couldn', 'between', 'you', "hadn't", "aren't", 'not', 'some', 'into', 'both', 'yourself', 'too', 'just', 'won', 'while', "doesn't", 'no', 'had', 'shan', 'our', 'hasn', 'himself', 'didn', 'don', 'about', 'wasn', 'further', 'an', 'more', 'mustn', 'are', 'doesn', 'over', 'or', 'my', 'through', 'nor', 'isn', 'up', 'theirs', "you've", 'now', 'was', 'be', 'did', 'were', 'doing', "mustn't", "needn't", 'his', 'before', 'it', 'themselves', 'mightn', 'a', 'aren', 'the', 'ours', 'at', 'because', 'here', 'during', 'her', "you'd", 'how', "wasn't", "you'll", 'myself', 'weren', "don't", "couldn't", 'being', "wouldn't", 'but', 'few', "hasn't", "shouldn't", 'him', 'should', "didn't", "weren't", 'its', 'down', 'been', 'whom', 'is', 'and', 'there', 'i', 'where', "she's", 'herself', 'hadn', 'am', 'once', 't', 'with', 'o', 'i

In [ ]:
#tokenize text
total_data['tokenized_text']=total_data['text'].apply(gensim.utils.simple_preprocess)

In [ ]:
total_data.head()

,id,text,target,tokenized_text
0,1,Our Deeds are the Reason of this #earthquake M...,1.0,"[our, deeds, are, the, reason, of, this, earth..."
1,4,Forest fire near La Ronge Sask. Canada,1.0,"[forest, fire, near, la, ronge, sask, canada]"
2,5,All residents asked to 'shelter in place' are ...,1.0,"[all, residents, asked, to, shelter, in, place..."
3,6,"13,000 people receive #wildfires evacuation or...",1.0,"[people, receive, wildfires, evacuation, order..."
4,7,Just got sent this photo from Ruby #Alaska as ...,1.0,"[just, got, sent, this, photo, from, ruby, ala..."


In [ ]:
# remove stopwords
def remove_stopwords(s):
    cleantext=[]
    for i in range(len(s)): 
        if(s[i] not in stop_words):
                cleantext.append(s[i])
    return cleantext


In [ ]:
total_data['clean_tok_text']=total_data['tokenized_text'].apply(lambda x:remove_stopwords(x))
total_data.head()

,id,text,target,tokenized_text,clean_tok_text
0,1,Our Deeds are the Reason of this #earthquake M...,1.0,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv..."
1,4,Forest fire near La Ronge Sask. Canada,1.0,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]"
2,5,All residents asked to 'shelter in place' are ...,1.0,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o..."
3,6,"13,000 people receive #wildfires evacuation or...",1.0,"[people, receive, wildfires, evacuation, order...","[people, receive, wildfires, evacuation, order..."
4,7,Just got sent this photo from Ruby #Alaska as ...,1.0,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi..."


In [ ]:
total_data.shape

(10876, 5)

In [ ]:
# join clean tokenized text
def join_fun(text):
    string=" "
    ans=string.join(text)
    return ans
total_data['final_text']=total_data['clean_tok_text'].apply(lambda x:join_fun(x))
total_data.head()

,id,text,target,tokenized_text,clean_tok_text,final_text
0,1,Our Deeds are the Reason of this #earthquake M...,1.0,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...",deeds reason earthquake may allah forgive us
1,4,Forest fire near La Ronge Sask. Canada,1.0,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada
2,5,All residents asked to 'shelter in place' are ...,1.0,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...",residents asked shelter place notified officer...
3,6,"13,000 people receive #wildfires evacuation or...",1.0,"[people, receive, wildfires, evacuation, order...","[people, receive, wildfires, evacuation, order...",people receive wildfires evacuation orders cal...
4,7,Just got sent this photo from Ruby #Alaska as ...,1.0,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...",got sent photo ruby alaska smoke wildfires pou...


# Predictions using Count Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

vec=cv.fit_transform(total_data["final_text"])

In [ ]:
print(total_data.shape,vec.shape)

(10876, 6) (10876, 28319)


In [ ]:
X=vec[:len(train_data)]
y=train_data.target
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.90)

In [ ]:
model=RandomForestClassifier(n_estimators=65)
model.fit(X_train,y_train)
pred=model.predict(X_test)
accuracy_score(pred,y_test)

0.7650918635170604

# Using TfIdf Vectorizer
TF-IDF is better than Count Vectorizers because it not only focuses on the frequency of words present in the corpus but also provides the importance of the words. We can then remove the words that are less important for analysis, hence making the model building less complex by reducing the input dimensions.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec=TfidfVectorizer()

vec1=tfidfvec.fit_transform(total_data["final_text"])

In [ ]:
X=vec1[:len(train_data)]
y=train_data.target
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.90)

In [ ]:
model=RandomForestClassifier(n_estimators=65)
model.fit(X_train,y_train)
pred=model.predict(X_test)
accuracy_score(pred,y_test)

0.7887139107611548

In [ ]:
test_vac=vec[len(train_data):]
test_vac

<3263x28319 sparse matrix of type '<class 'numpy.int64'>'
	with 35775 stored elements in Compressed Sparse Row format>

# BertTokenizier


In [ ]:
from transformers import BertTokenizer, TFBertModel, TFAutoModel,AutoTokenizer
model_name ='joeddav/xlm-roberta-large-xnli'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

In [ ]:
def encode_sentence(s):
    tokens=[]
    tokens.append('[CLS]')
    tokens+=list(tokenizer.tokenize(s))
#     print(tokens)
    return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
main_data=total_data['final_text']
main_data=pd.DataFrame(main_data)

In [ ]:
main_data.head()

,final_text
0,deeds reason earthquake may allah forgive us
1,forest fire near la ronge sask canada
2,residents asked shelter place notified officer...
3,people receive wildfires evacuation orders cal...
4,got sent photo ruby alaska smoke wildfires pou...


In [ ]:
encode_sentence(main_data['final_text'][0])

[3, 8, 297, 7, 31635, 109270, 13722, 350, 1543, 6, 40077, 100, 96259, 1821]

In [ ]:
tokenized=[]
for i in range(len(main_data)):
    tokenized.append(encode_sentence(main_data['final_text'][i]))
main_data['tokenized']=tokenized

In [ ]:
main_data['target']=total_data['target']
main_data

,final_text,tokenized,target
0,deeds reason earthquake may allah forgive us,"[3, 8, 297, 7, 31635, 109270, 13722, 350, 1543...",1.0
1,forest fire near la ronge sask canada,"[3, 100, 525, 11476, 43573, 21, 107629, 13, 57...",1.0
2,residents asked shelter place notified officer...,"[3, 160641, 37170, 2412, 1277, 56, 3687, 959, ...",1.0
3,people receive wildfires evacuation orders cal...,"[3, 3395, 53299, 56409, 73702, 7, 166499, 1363...",1.0
4,got sent photo ruby alaska smoke wildfires pou...,"[3, 4163, 9325, 16186, 21922, 53, 10, 68465, 2...",1.0
...,...,...,...
10871,earthquake safety los angeles ûò safety fasten...,"[3, 109270, 13722, 350, 81900, 388, 61909, 90,...",NaN
10872,storm ri worse last hurricane city others hard...,"[3, 77076, 1427, 124632, 4568, 3587, 12056, 86...",NaN
10873,green line derailment chicago http co utbxlcbiuy,"[3, 50997, 13315, 8, 68823, 674, 87904, 519, 1...",NaN
10874,meg issues hazardous weather outlook hwo http ...,"[3, 721, 37348, 141033, 10821, 92949, 6, 20646...",NaN


# Attention Mask and Token Type ID:

In [ ]:
# see the documentations

mask=[]
for i in range(len(main_data)):
    padded_seq=tokenizer(main_data['final_text'][i],padding=True,add_special_tokens =True)
    mask.append(padded_seq)

main_data['masked']=mask
main_data.head()

,final_text,tokenized,target,masked
0,deeds reason earthquake may allah forgive us,"[3, 8, 297, 7, 31635, 109270, 13722, 350, 1543...",1.0,"[input_ids, attention_mask]"
1,forest fire near la ronge sask canada,"[3, 100, 525, 11476, 43573, 21, 107629, 13, 57...",1.0,"[input_ids, attention_mask]"
2,residents asked shelter place notified officer...,"[3, 160641, 37170, 2412, 1277, 56, 3687, 959, ...",1.0,"[input_ids, attention_mask]"
3,people receive wildfires evacuation orders cal...,"[3, 3395, 53299, 56409, 73702, 7, 166499, 1363...",1.0,"[input_ids, attention_mask]"
4,got sent photo ruby alaska smoke wildfires pou...,"[3, 4163, 9325, 16186, 21922, 53, 10, 68465, 2...",1.0,"[input_ids, attention_mask]"


# Train Model

In [ ]:
max_len=32
def build_model():
    bert_encoder = TFAutoModel.from_pretrained('joeddav/xlm-roberta-large-xnli')
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    embedding = bert_encoder([input_word_ids, input_mask])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
def input_convert(data):
    inputs={
        'input_word_ids':[],
        'input_mask':[]
    }
    for each in data:
        inputs['input_word_ids'].append(each['input_ids'])
        inputs['input_mask'].append(each['attention_mask'])
        
    
    inputs['input_word_ids']= tf.ragged.constant( inputs['input_word_ids']).to_tensor()
    inputs['input_mask']= tf.ragged.constant( inputs['input_mask']).to_tensor()
    return inputs

In [ ]:

data_for_predictions=main_data[len(train_data):]
data_for_fitting=main_data[:len(train_data)]
data_for_fitting
    

,final_text,tokenized,target,masked
0,deeds reason earthquake may allah forgive us,"[3, 8, 297, 7, 31635, 109270, 13722, 350, 1543...",1.0,"[input_ids, attention_mask]"
1,forest fire near la ronge sask canada,"[3, 100, 525, 11476, 43573, 21, 107629, 13, 57...",1.0,"[input_ids, attention_mask]"
2,residents asked shelter place notified officer...,"[3, 160641, 37170, 2412, 1277, 56, 3687, 959, ...",1.0,"[input_ids, attention_mask]"
3,people receive wildfires evacuation orders cal...,"[3, 3395, 53299, 56409, 73702, 7, 166499, 1363...",1.0,"[input_ids, attention_mask]"
4,got sent photo ruby alaska smoke wildfires pou...,"[3, 4163, 9325, 16186, 21922, 53, 10, 68465, 2...",1.0,"[input_ids, attention_mask]"
...,...,...,...,...
7608,two giant cranes holding bridge collapse nearb...,"[3, 6626, 6051, 18, 6, 15182, 1444, 104064, 6,...",1.0,"[input_ids, attention_mask]"
7609,aria_ahrary thetawniest control wild fires cal...,"[3, 6, 6232, 454, 11, 11894, 1294, 70, 102, 63...",1.0,"[input_ids, attention_mask]"
7610,utc km volcano hawaii http co zdtoyd ebj,"[3, 486, 238, 1785, 2741, 38938, 19896, 1573, ...",1.0,"[input_ids, attention_mask]"
7611,police investigating bike collided car little ...,"[3, 35206, 32603, 1916, 15528, 61770, 48141, 2...",1.0,"[input_ids, attention_mask]"


In [ ]:
train_input=input_convert(data_for_fitting['masked'].values)
for key in train_input.keys():
    train_input[key] = train_input[key][:,:max_len]

In [ ]:
train_input

{'input_word_ids': <tf.Tensor: shape=(7613, 32), dtype=int32, numpy=
 array([[     0,      8,    297, ...,      0,      0,      0],
        [     0,    100,    525, ...,      0,      0,      0],
        [     0, 160641,  37170, ...,      0,      0,      0],
        ...,
        [     0,    486,    238, ...,      0,      0,      0],
        [     0,  35206,  32603, ...,      0,      0,      0],
        [     0,  42850,  44556, ...,      0,      0,      0]], dtype=int32)>,
 'input_mask': <tf.Tensor: shape=(7613, 32), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
early_stop=tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
with strategy.scope():
    model=build_model()
    model.summary()
    model.fit(train_input,main_data['target'].values,epochs=5,verbose=1,batch_size=128,validation_split=0.1,callbacks=[early_stop])

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some layers from the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing TFXLMRobertaModel: ['classifier']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at joeddav/xlm-roberta-large-xnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 32)]         0                                            
__________________________________________________________________________________________________
tfxlm_roberta_model (TFXLMRober TFBaseModelOutputWit 559890432   input_word_ids[0][0]             
                                                                 input_mask[0][0]                 
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 1024)         0           tfxlm_roberta_model[0][0]    

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


54/54 [==============================] - 240s 2s/step - loss: 0.6837 - accuracy: 0.6249 - val_loss: 0.4140 - val_accuracy: 0.8281
Epoch 2/5
54/54 [==============================] - 10s 183ms/step - loss: 0.4254 - accuracy: 0.8247 - val_loss: 0.4805 - val_accuracy: 0.7966
Epoch 3/5
54/54 [==============================] - 10s 183ms/step - loss: 0.4056 - accuracy: 0.8294 - val_loss: 0.4478 - val_accuracy: 0.8202
Epoch 4/5
54/54 [==============================] - 10s 183ms/step - loss: 0.3632 - accuracy: 0.8547 - val_loss: 0.4391 - val_accuracy: 0.8241


# predictions

In [ ]:
test_input=input_convert(data_for_predictions['masked'].values)
for key in train_input.keys():
    test_input[key] = test_input[key][:,:max_len]

predictions=[np.argmax(i) for i in model.predict(test_input)]

In [ ]:
submission['target']=predictions
submission.to_csv("submission.csv",index=False)

In [ ]:
submission.head(10)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0
